In [1]:
import math, subprocess
import pandas as pd
import numpy as np
from IPython.display import display, clear_output

In [2]:
def correr_experimento(cant_threads, maximo, lista_archivos):
    # Crear proceso para ejecutar el codigo.
    process = subprocess.Popen(["../build/ContarPalabras", cant_threads, maximo] + lista_archivos, stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE, universal_newlines = True)

    # Correr experimento.
    exit_code = process.wait()

    # Verificar que el proceso no fallo.
    if exit_code != 0: raise ValueError(F"Hubo un error en la experimentacion para la cantidad de threads: {cant_threads} con la instancia {lista_archivos}.")
    
    # Leer salida de STDERR con los tiempos de ejecucion de cada metodo.
    tiempo_de_ejecucion = float(process.stderr.read());

    # Leo de STDOUT el tiempo del maximo paralelo si hay
    tiempo_max_paralelo = 0.0
    if (maximo == "true"):
        tiempo_max_paralelo = float(process.stdout.read());
    
    process.stdin.close();
    process.stdout.close();
    process.stderr.close();
    
    return tiempo_de_ejecucion, tiempo_max_paralelo


### Experimento 1


In [3]:
experimentos = [];
df_caso_random = pd.read_csv("instancias/random/indice.txt");
cantThreads = 19
#lista_archivos_string = ""
#with open(F"instancias/random/indice.txt", 'r') as f:
#    lista_archivos_string = f.read();
lista_archivos = df_caso_random.stack().tolist()
for threads in range(1, cantThreads):
    experimentos.append(["big_random", str(threads), "true", lista_archivos])

In [ ]:
columnas = ["dataset", "cant_threads", "metodo", "tiempo_carga", "tiempo_max"];
filas = [];
numero = 1
T = 5 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
for experimento in experimentos:
    # Voy mostrando que experimento se está ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos)))
    numero += 1
    
    # Ejecutamos el experimento T veces y obtenemos la mediana.
    tiempos_carga = []
    tiempos_max = []
    
    for _ in range(0, T):
        tiempo_carga, tiempo_max = correr_experimento(experimento[1], experimento[2], experimento[3])
        tiempos_carga.append(tiempo_carga);
        tiempos_max.append(tiempo_max);
    tiempo_carga = np.median(tiempos_carga);
    tiempo_max = np.median(tiempos_max);
    filas.append([experimento[0], experimento[1], experimento[2], tiempo_carga, tiempo_max]);
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("resultados/exp1.csv", index=False, header=True);

'Experimento: 3/18'

### Experimento 1.2

In [3]:
experimentos = [];
df_caso_randomNoUNiforme = pd.read_csv("instancias/randomNoUniforme/indice.txt");
cantThreads = 19
#lista_archivos_string = ""
#with open(F"instancias/random/indice.txt", 'r') as f:
#    lista_archivos_string = f.read();
lista_archivos = df_caso_randomNoUNiforme.stack().tolist()
for threads in range(1, cantThreads):
    experimentos.append(["random_noUniforme", str(threads), "true", lista_archivos])

In [4]:
columnas = ["dataset", "cant_threads", "metodo", "tiempo_carga", "tiempo_max","varianza_carga","varianza_max"];
filas = [];
numero = 1
T = 5 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
for experimento in experimentos:
    # Voy mostrando que experimento se está ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos)))
    numero += 1
    
    # Ejecutamos el experimento T veces y obtenemos la mediana.
    tiempos_carga = []
    tiempos_max = []
    
    for _ in range(0, T):
        tiempo_carga, tiempo_max = correr_experimento(experimento[1], experimento[2], experimento[3])
        tiempos_carga.append(tiempo_carga);
        tiempos_max.append(tiempo_max);
    tiempo_carga = np.median(tiempos_carga);
    tiempo_max = np.median(tiempos_max);
    varianza_carga = np.var(tiempos_carga)
    varianza_max = np.var(tiempos_max)
    filas.append([experimento[0], experimento[1], experimento[2], tiempo_carga, tiempo_max, varianza_carga, varianza_max]);
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("resultados/exp1.csv", index=False, header=True);

'Experimento: 3/18'

KeyboardInterrupt: 

## Experimento 2

In [15]:
cantThreads = 20
experimentos = []
df_indice_directorios = pd.read_csv("instancias/separados/indice_directorios.txt")

for directorio in df_indice_directorios.stack().to_list():
    lista_archivos = pd.read_csv(directorio+"/indice.txt").stack().tolist()
    
    for threads in range(1, cantThreads):
        experimentos.append([directorio, str(threads), lista_archivos])

In [16]:
columnas = ["dataset", "cant_threads", "tiempo_carga"];
filas = [];
numero = 1
T = 4 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
for experimento in experimentos:
    # Voy mostrando que experimento se está ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos)))
    numero += 1
    
    # Ejecutamos el experimento T veces y obtenemos la mediana.
    tiempos_carga = []
    for _ in range(0, T):
        tiempo_carga, _ = correr_experimento(cant_threads=experimento[1],maximo="False",lista_archivos=experimento[2])
        tiempos_carga.append(tiempo_carga);
    tiempo_carga = np.median(tiempos_carga);
    filas.append([experimento[0], experimento[1], tiempo_carga]);
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("resultados/experimento_separados.csv", index=False, header=True);

'Experimento: 1/190'

ValueError: could not convert string to float: ''

## Experimento 3

In [7]:
'''
cantThreads = 19
experimentos = []
df_indice_directorios = pd.read_csv("instancias/multiplesAlfabetos/indice.txt")

for directorio in df_indice_directorios.stack().to_list():
    lista_archivos = pd.read_csv(directorio+"/indice.txt").stack().tolist()
    
    for threads in range(1, cantThreads):
        experimentos.append([directorio, str(threads), lista_archivos])
'''

In [ ]:
'''
columnas = ["dataset", "cant_threads", "tiempo_carga", "tiempo_max", "varianza_carga", "varianza_max"];
filas = [];
numero = 1
T = 5 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
for experimento in experimentos:
    # Voy mostrando que experimento se está ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos)))
    numero += 1
    
    # Ejecutamos el experimento T veces y obtenemos la mediana.
    tiempos_carga = []
    tiempos_max = []
    for _ in range(0, T):
        tiempo_carga, tiempo_max = correr_experimento(cant_threads=experimento[1],maximo="true",lista_archivos=experimento[2])
        tiempos_carga.append(tiempo_carga)
        tiempos_max.append(tiempo_max)
    tiempo_carga = np.median(tiempos_carga)
    tiempo_max = np.median(tiempos_max)
    varianza_carga = np.var(tiempos_carga)
    varianza_max = np.var(tiempos_max)
    filas.append([experimento[0], experimento[1], tiempo_carga, tiempo_max, varianza_carga, varianza_max]);
df_resultado = pd.DataFrame(filas, columns=columnas)
df_resultado.to_csv("resultados/experimento_multiplesAlfabetos.csv", index=False, header=True)
'''

'\ncolumnas = ["dataset", "cant_threads", "tiempo_carga", "tiempo_max", "varianza_carga", "varianza_max"];\nfilas = [];\nnumero = 1\nT = 5 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).\nfor experimento in experimentos:\n    # Voy mostrando que experimento se está ejecutando.\n    clear_output(wait=True)\n    display(\'Experimento: \' + str(numero) + "/" + str(len(experimentos)))\n    numero += 1\n    \n    # Ejecutamos el experimento T veces y obtenemos la mediana.\n    tiempos_carga = []\n    tiempos_max = []\n    for _ in range(0, T):\n        tiempo_carga, tiempo_max = correr_experimento(cant_threads=experimento[1],maximo="true",lista_archivos=experimento[2])\n        tiempos_carga.append(tiempo_carga)\n        tiempos_max.append(tiempo_max)\n    tiempo_carga = np.median(tiempos_carga)\n    tiempo_max = np.median(tiempos_max)\n    varianza_carga = np.var(tiempos_carga)\n    varianza_max = np.var(tiempos_max)\n    filas.append([experimento[0], e

## Experimento 3.1

In [9]:
experimentos = [];
df_alfabeto2 = pd.read_csv("instancias/multiplesAlfabetos/alfabeto-2/indice.txt");
cantThreads = 19
#lista_archivos_string = ""
#with open(F"instancias/random/indice.txt", 'r') as f:
#    lista_archivos_string = f.read();
lista_archivos = df_alfabeto2.stack().tolist()
for threads in range(1, cantThreads):
    experimentos.append(["random_alfabeto2", str(threads), "true", lista_archivos])

In [10]:
columnas = ["dataset", "cant_threads", "metodo", "tiempo_carga", "tiempo_max","varianza_carga","varianza_max"];
filas = [];
numero = 1
T = 5 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
for experimento in experimentos:
    # Voy mostrando que experimento se está ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos)))
    numero += 1
    
    # Ejecutamos el experimento T veces y obtenemos la mediana.
    tiempos_carga = []
    tiempos_max = []
    
    for _ in range(0, T):
        tiempo_carga, tiempo_max = correr_experimento(experimento[1], experimento[2], experimento[3])
        tiempos_carga.append(tiempo_carga);
        tiempos_max.append(tiempo_max);
    tiempo_carga = np.median(tiempos_carga);
    tiempo_max = np.median(tiempos_max);
    varianza_carga = np.var(tiempos_carga)
    varianza_max = np.var(tiempos_max)
    filas.append([experimento[0], experimento[1], experimento[2], tiempo_carga, tiempo_max, varianza_carga, varianza_max]);
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("resultados/exp1.csv", index=False, header=True);

'Experimento: 5/18'

KeyboardInterrupt: 

## Experimento 3.2

In [11]:
experimentos = [];
df_alfabeto1 = pd.read_csv("instancias/multiplesAlfabetos/alfabeto-1/indice.txt");
cantThreads = 19
#lista_archivos_string = ""
#with open(F"instancias/random/indice.txt", 'r') as f:
#    lista_archivos_string = f.read();
lista_archivos = df_alfabeto1.stack().tolist()
for threads in range(1, cantThreads):
    experimentos.append(["random_alfabeto1", str(threads), "true", lista_archivos])

In [12]:
columnas = ["dataset", "cant_threads", "metodo", "tiempo_carga", "tiempo_max","varianza_carga","varianza_max"];
filas = [];
numero = 1
T = 5 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
for experimento in experimentos:
    # Voy mostrando que experimento se está ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos)))
    numero += 1
    
    # Ejecutamos el experimento T veces y obtenemos la mediana.
    tiempos_carga = []
    tiempos_max = []
    
    for _ in range(0, T):
        tiempo_carga, tiempo_max = correr_experimento(experimento[1], experimento[2], experimento[3])
        tiempos_carga.append(tiempo_carga);
        tiempos_max.append(tiempo_max);
    tiempo_carga = np.median(tiempos_carga);
    tiempo_max = np.median(tiempos_max);
    varianza_carga = np.var(tiempos_carga)
    varianza_max = np.var(tiempos_max)
    filas.append([experimento[0], experimento[1], experimento[2], tiempo_carga, tiempo_max, varianza_carga, varianza_max]);
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("resultados/exp1.csv", index=False, header=True);

'Experimento: 4/18'

KeyboardInterrupt: 

## Experimento 3.3

In [13]:
experimentos = [];
df_alfabeto0 = pd.read_csv("instancias/multiplesAlfabetos/alfabeto-0/indice.txt");
cantThreads = 19
#lista_archivos_string = ""
#with open(F"instancias/random/indice.txt", 'r') as f:
#    lista_archivos_string = f.read();
lista_archivos = df_alfabeto0.stack().tolist()
for threads in range(1, cantThreads):
    experimentos.append(["random_alfabeto0", str(threads), "true", lista_archivos])

In [14]:
columnas = ["dataset", "cant_threads", "metodo", "tiempo_carga", "tiempo_max","varianza_carga","varianza_max"];
filas = [];
numero = 1
T = 5 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
for experimento in experimentos:
    # Voy mostrando que experimento se está ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos)))
    numero += 1
    
    # Ejecutamos el experimento T veces y obtenemos la mediana.
    tiempos_carga = []
    tiempos_max = []
    
    for _ in range(0, T):
        tiempo_carga, tiempo_max = correr_experimento(experimento[1], experimento[2], experimento[3])
        tiempos_carga.append(tiempo_carga);
        tiempos_max.append(tiempo_max);
    tiempo_carga = np.median(tiempos_carga);
    tiempo_max = np.median(tiempos_max);
    varianza_carga = np.var(tiempos_carga)
    varianza_max = np.var(tiempos_max)
    filas.append([experimento[0], experimento[1], experimento[2], tiempo_carga, tiempo_max, varianza_carga, varianza_max]);
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("resultados/exp1.csv", index=False, header=True);

'Experimento: 4/18'

KeyboardInterrupt: 